## Bläddra ner till figurerna

In [ ]:
######################## IMPORTS, FUNCTIONS, TEAMS, DRIVERS, COLORS ETC ########################

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from scipy.stats import rankdata
from f1_scoring_2023 import f1_parameters

# Check if running headless and prepare for file output
headless = os.getenv('HEADLESS', 'False').lower() in ['true', '1']
output_dir = os.getenv('FIG_OUTPUT_DIR', '.')

def show_or_save(plt, name):
  if headless:
    plt.savefig(f'{output_dir}/{name}.png')
  else:
    plt.show()

def check_doubles(in_array):
    result = []
    n = len(in_array)
    for i in range(n):
        count = 0
        # only look at number preceding the one at i
        for j in range(i): 
            if in_array[i] == in_array[j]:
                count += 1
                # we look for cases where there is exactly one match
                # so we know that at i it is the 2nd occurrence
                if count == 2:
                    break
        if count == 1:
            result.append(in_array[i])
    return np.asarray(result)


#Team names, driver names, and colors
teams = ['Mercedes', 'RedBull', 'McLaren', 'Aston Martin', 'Alpine', 'Ferarri',  'AlphaTauri' , 'Alfa Romeo', 'Haas',  'Williams']
drivers = ['HAM', 'RUS', 'PER', 'VER', 'NOR', 'PIA', 'ALO', 'STR', 'OCO', 'GAS', 'LEC', 'SAI', 'DEV',\
                    'TSU', 'BOT', 'ZHO', 'MAG', 'HUL', 'SAR', 'ALB']

color_dict_driver = {'HAM': '#00D2BE', 'RUS': '#00D2BE', 'PER': '#0600EF', 'VER': '#0600EF', 'NOR': '#FF8700', 'PIA': '#FF8700',\
 'ALO' : '#006F62',  'STR' : '#006F62', 'OCO' : '#0090FF', 'GAS' : '#0090FF', 'LEC' : '#DC0000',  'SAI' : '#DC0000', 'DEV' : '#2B4562' \
  , 'TSU' : '#2B4562' , 'BOT' : '#900000', 'ZHO' : '#900000', 'MAG' : '#000000', 'HUL' : '#000000','SAR' : '#005AFF', 'ALB' : '#005AFF'  }

color_dict_teams = {'Mercedes': '#00D2BE', 'RedBull': '#0600EF', 'McLaren': '#FF8700',  'Aston Martin' : '#006F62', \
 'Alpine' : '#0090FF', 'Ferarri' : '#DC0000', 'AlphaTauri' : '#2B4562' , 'Alfa Romeo' : '#900000', 'Haas' : '#000000', 'Williams' : '#005AFF' }

df_driver_ranking =  pd.DataFrame( columns = drivers)
df_constr_points = pd.DataFrame( columns = teams)
df_constr_ranking = pd.DataFrame(columns = teams)
df_driver_bet_result = pd.DataFrame( columns = ['Jesper', 'Jonas', 'Lindskog', 'Mattias', 'Jonas_Far', 'Edvard*'])
df_constr_bet_result = pd.DataFrame( columns = ['Jesper', 'Jonas', 'Lindskog', 'Mattias', 'Jonas_Far', 'Edvard*'])





#Import scoring, bet, modifications
df_driver_bet, df_constructor_bet, df_driver_points, df_driver_mod, df_constr_mod, tracks = f1_parameters(drivers, teams)


########################  CALCULATIONS  ########################



#Dataframe for driver points, cumulative
df_cum_driver_points = df_driver_points.cumsum(axis =0)

#Loop over the tracks and set driver ranking, bet results, construcor points, ranking bet etc
for tr in tracks:
    
    #Driver specific calcualtions
    driver_ranking = (len(df_cum_driver_points.loc[tr].values)+1 -  rankdata(df_cum_driver_points.loc[tr], method = 'min'))
    doubles_driv = check_doubles(driver_ranking)
    for d in doubles_driv:
        driver_ranking[driver_ranking == d] = df_driver_mod.loc[tr].values[driver_ranking == d]

    df_driver_ranking.loc[tr] = driver_ranking

    df_driver_bet_result.loc[tr] = np.asarray(
                                               (np.abs(df_driver_ranking.loc[tr] - df_driver_bet.loc['Jesper']).sum(),\
                                               np.abs(df_driver_ranking.loc[tr] - df_driver_bet.loc['Jonas']).sum(), \
                                               np.abs(df_driver_ranking.loc[tr] - df_driver_bet.loc['Lindskog']).sum(), \
                                               np.abs(df_driver_ranking.loc[tr] - df_driver_bet.loc['Mattias']).sum(), \
                                               np.abs(df_driver_ranking.loc[tr] - df_driver_bet.loc['Jonas_Far']).sum(), \
                                               np.abs(df_driver_ranking.loc[tr] - df_driver_bet.loc['Edvard*']).sum())
                                                )

    #Constructor specific calculations
    merc_p = df_cum_driver_points.loc[tr].HAM + df_cum_driver_points.loc[tr].RUS
    red_p = df_cum_driver_points.loc[tr].PER + df_cum_driver_points.loc[tr].VER
    fer_p = df_cum_driver_points.loc[tr].LEC + df_cum_driver_points.loc[tr].SAI
    aston_p = df_cum_driver_points.loc[tr].ALO + df_cum_driver_points.loc[tr].STR
    mclaren_p = df_cum_driver_points.loc[tr].PIA + df_cum_driver_points.loc[tr].NOR
    alpine_p = df_cum_driver_points.loc[tr].GAS + df_cum_driver_points.loc[tr].OCO
    alpha_p = df_cum_driver_points.loc[tr].DEV + df_cum_driver_points.loc[tr].TSU
    haas_p = df_cum_driver_points.loc[tr].HUL + df_cum_driver_points.loc[tr].MAG
    alfarom_p = df_cum_driver_points.loc[tr].BOT + df_cum_driver_points.loc[tr].ZHO
    will_p = df_cum_driver_points.loc[tr].ALB + df_cum_driver_points.loc[tr].SAR

    df_constr_points.loc[tr] = [merc_p, red_p,  mclaren_p, aston_p, alpine_p, fer_p, alpha_p,  alfarom_p, haas_p, will_p]


    constr_ranking =  (len(df_constr_points.loc[tr].values)+1 -  rankdata(df_constr_points.loc[tr], method = 'min'))
    doubles_constr = check_doubles(constr_ranking)
    
    for d in doubles_constr:
        constr_ranking[constr_ranking == d] = df_constr_mod.loc[tr].values[constr_ranking == d]
    
    df_constr_ranking.loc[tr] = constr_ranking
    df_constr_bet_result.loc[tr] = np.asarray(
                                            (np.abs(df_constr_ranking.loc[tr] -df_constructor_bet.loc['Jesper']).sum(), \
                                             np.abs(df_constr_ranking.loc[tr] -df_constructor_bet.loc['Jonas']).sum(), \
                                             np.abs(df_constr_ranking.loc[tr] -df_constructor_bet.loc['Lindskog']).sum(),\
                                             np.abs(df_constr_ranking.loc[tr] -df_constructor_bet.loc['Mattias']).sum(),\
                                             np.abs(df_constr_ranking.loc[tr] -df_constructor_bet.loc['Jonas_Far']).sum(),\
                                             np.abs(df_constr_ranking.loc[tr] -df_constructor_bet.loc['Edvard*']).sum())
                                             )
    



In [ ]:
num_race = len(df_cum_driver_points.index.values)

########################  PLOTTING  ########################

fontP = FontProperties()
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('legend', fontsize=16)

#Plot the cumulative points for drivers
fontP.set_size('large')
df_cum_driver_points.plot(color=[color_dict_driver.get(x) for x in color_dict_driver])
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', prop=fontP)
fig = plt.gcf()
fig.set_size_inches(14, 9)
ax = plt.gca()
ax.set_xticks(np.arange(0,num_race))
ax.set_xticklabels(list(df_cum_driver_points.index), rotation = 20, fontsize=16)

for i, txt in enumerate(list(drivers)):
    ax.annotate(txt, (num_race-1, df_cum_driver_points.iloc[-1].values[i]))
plt.title('Driver points, cumulative', fontsize=20)
show_or_save(plt, 'cum_driver_points')


#plot Driver ranking
df_driver_ranking.plot(color=[color_dict_driver.get(x) for x in color_dict_driver])
fig = plt.gcf()
fig.set_size_inches(14, 9)
ax = plt.gca()
ax.invert_yaxis()
ax.set_xticks(np.arange(0,num_race))
ax.set_xticklabels(list(df_cum_driver_points.index), rotation = 20, fontsize=16)
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', prop=fontP)
plt.title('Driver ranking', fontsize=20)
for i, txt in enumerate(list(drivers)):
    ax.annotate(txt, (num_race-1, df_driver_ranking.iloc[-1].values[i]))
show_or_save(plt, 'driver_ranking')


#plot constructor points, cumulative
df_constr_points.plot(color=[color_dict_teams.get(x) for x in color_dict_teams])
fig = plt.gcf()
fig.set_size_inches(14, 9)
ax = plt.gca()
ax.set_xticks(np.arange(0,num_race))
ax.set_xticklabels(list(df_cum_driver_points.index), rotation = 20, fontsize=16)
plt.title('Constructor points, cumulative', fontsize=20)
show_or_save(plt, 'cum_constructor_points')

#plot constructor ranking
df_constr_ranking.plot(color=[color_dict_teams.get(x) for x in color_dict_teams])
fig = plt.gcf()
fig.set_size_inches(14, 9)
ax = plt.gca()
ax.set_xticks(np.arange(0,num_race))
ax.set_xticklabels(list(df_cum_driver_points.index), rotation = 20, fontsize=16)
ax.invert_yaxis()
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', prop=fontP)
plt.title('Constructor ranking', fontsize=20)
show_or_save(plt, 'constructor_ranking')

#plot driver bet result
df_driver_bet_result.plot()
fig = plt.gcf()
fig.set_size_inches(14, 9)
#plt.ylim([0, 60])
ax = plt.gca()
ax.set_xticks(np.arange(0,num_race))
ax.set_xticklabels(list(df_cum_driver_points.index), rotation = 20, fontsize=16)
plt.legend(loc='upper right')
plt.title('Driver betting', fontsize=20)
show_or_save(plt, 'driver_betting')

#plot constructor bet result
df_constr_bet_result.plot()
fig = plt.gcf()
fig.set_size_inches(14, 9)
#plt.ylim([0, 15])
ax = plt.gca()
ax.set_xticks(np.arange(0,num_race))
ax.set_xticklabels(list(df_cum_driver_points.index), rotation = 20, fontsize=16)
plt.legend(loc='upper right')
plt.title('Constructor betting', fontsize=20)
show_or_save(plt, 'constructor_betting')



"""fig = plt.gcf()
fig.set_size_inches(14, 9)
np.abs(df_driver_ranking.loc[tr] -df_driver_bet.loc['Jesper']).plot(marker = '.', label  = 'Jesper')
np.abs(df_driver_ranking.loc[tr] -df_driver_bet.loc['Jonas']).plot(marker = '.', label  = 'Jonas')
np.abs(df_driver_ranking.loc[tr] -df_driver_bet.loc['Lindskog']).plot(marker = '.', label  = 'Lindskog')
plt.gca().set_xticks(np.linspace(0,19,20))
plt.gca().set_xticklabels(drivers)
plt.legend(loc='upper right')
plt.gca().set_ylabel('Points given by driver', fontsize=16)
show_or_save(plt)"""

df_driver_bet_points = pd.concat([df_driver_ranking.iloc[[-1]], df_driver_bet])
as_list = df_driver_bet_points.index.tolist()
idx = as_list.index(df_driver_ranking.index[-1])
as_list[idx] = 'Current standing'
df_driver_bet_points.index = as_list
          
df_driver_bet_points.plot(color=[color_dict_driver.get(x) for x in color_dict_driver], style='*', legend = False)
fig = plt.gcf()
fig.set_size_inches(14, 9)
ax = plt.gca()
ax.invert_yaxis()
ax.set_xticks(np.arange(0,7))
# Set ticks labels for x-axis
ax.set_xticklabels(as_list, rotation = 20, fontsize=16)


for i, txt in enumerate(list(drivers)):
    ax.annotate(txt, (-0.25, df_driver_ranking.iloc[-1].values[i]))
    ax.annotate(txt, (0.71, df_driver_bet.iloc[0].values[i]))
    ax.annotate(np.abs(df_driver_ranking.loc[tr] -df_driver_bet.loc['Jesper'])[i], (1.09, df_driver_bet.iloc[0].values[i]))
    ax.annotate(txt, (1.71, df_driver_bet.iloc[1].values[i]))
    ax.annotate(np.abs(df_driver_ranking.loc[tr] -df_driver_bet.loc['Jonas'])[i], (2.09, df_driver_bet.iloc[1].values[i]))
    ax.annotate(txt, (2.71, df_driver_bet.iloc[2].values[i]))
    ax.annotate(np.abs(df_driver_ranking.loc[tr] -df_driver_bet.loc['Lindskog'])[i], (3.09, df_driver_bet.iloc[2].values[i]))
    ax.annotate(txt, (3.71, df_driver_bet.iloc[3].values[i]))
    ax.annotate(np.abs(df_driver_ranking.loc[tr] -df_driver_bet.loc['Mattias'])[i], (4.09, df_driver_bet.iloc[3].values[i]))
    ax.annotate(txt, (4.71, df_driver_bet.iloc[4].values[i]))
    ax.annotate(np.abs(df_driver_ranking.loc[tr] -df_driver_bet.loc['Jonas_Far'])[i], (5.09, df_driver_bet.iloc[4].values[i]))
    ax.annotate(txt, (5.71, df_driver_bet.iloc[-1].values[i]))
    ax.annotate(np.abs(df_driver_ranking.loc[tr] -df_driver_bet.loc['Edvard*'])[i], (6.09, df_driver_bet.iloc[-1].values[i]))

plt.title('Driver bet points', fontsize=20)
show_or_save(plt, 'driver_bet_points')
##


df_constructor_bet_points = pd.concat([df_constr_ranking.iloc[[-1]], df_constructor_bet])
df_constructor_bet_points.index = as_list
df_constructor_bet_points.plot(color=[color_dict_teams.get(x) for x in color_dict_teams], style='*', legend = False)
fig = plt.gcf()
fig.set_size_inches(14, 9)
ax = plt.gca()
ax.invert_yaxis()
ax.set_xticks(np.arange(0,7))
# Set ticks labels for x-axis
ax.set_xticklabels(as_list, rotation = 20, fontsize=16)

for i, txt in enumerate(list(teams)):
    ax.annotate(txt, (-0.25, df_constr_ranking.iloc[-1].values[i]))
    ax.annotate(txt, (0.45, df_constructor_bet.iloc[0].values[i]))
    ax.annotate(np.abs(df_constr_ranking.loc[tr] -df_constructor_bet.loc['Jesper'])[i], (1.09, df_constructor_bet.iloc[0].values[i]))
    ax.annotate(txt, (1.45, df_constructor_bet.iloc[1].values[i]))
    ax.annotate(np.abs(df_constr_ranking.loc[tr] -df_constructor_bet.loc['Jonas'])[i], (2.09, df_constructor_bet.iloc[1].values[i]))
    ax.annotate(txt, (2.45, df_constructor_bet.iloc[2].values[i]))
    ax.annotate(np.abs(df_constr_ranking.loc[tr] -df_constructor_bet.loc['Lindskog'])[i], (3.09, df_constructor_bet.iloc[2].values[i]))
    ax.annotate(txt, (3.45, df_constructor_bet.iloc[3].values[i]))
    ax.annotate(np.abs(df_constr_ranking.loc[tr] -df_constructor_bet.loc['Mattias'])[i], (4.09, df_constructor_bet.iloc[3].values[i]))
    ax.annotate(txt, (4.45, df_constructor_bet.iloc[4].values[i]))
    ax.annotate(np.abs(df_constr_ranking.loc[tr] -df_constructor_bet.loc['Jonas_Far'])[i], (5.09, df_constructor_bet.iloc[4].values[i]))
    ax.annotate(txt, (5.45, df_constructor_bet.iloc[-1].values[i]))
    ax.annotate(np.abs(df_constr_ranking.loc[tr] -df_constructor_bet.loc['Edvard*'])[i], (6.09, df_constructor_bet.iloc[-1].values[i]))

plt.title('Constructor bet points', fontsize=20)
show_or_save(plt, 'constructor_bet_points')

In [ ]:
print('Driver bet result')
print(df_driver_bet_result.iloc[-1])

In [ ]:
print('Constructor bet result')
print(df_constr_bet_result.iloc[-1])